In [1]:
import sys
import json
import glob
import pickle
sys.path.append('../')

import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import joblib
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline

from evaluate.lane import LaneEval
    
BASE_DIR = '../../logs/comma_benign/'


In [2]:
df_res = []

y_samples = np.arange(160, 711, 10)

for model_type in  ['laneatt', 'scnn', 'ultrafast', 'polylanenet']:
    print(model_type)
    res = []
    imgs = []
    for sc_n in range(1, 101):

        with open(BASE_DIR + f'logs_more_{model_type}/scb{sc_n}/replay/annotation.json') as f:
            gts = json.loads(f.read())

        for frame in range(20):

            gt = gts[frame]

            run_time = 0 #pred['run_time']
            _gt_lanes = gt['lanes']
            y = gt['h_samples']
            y = np.round((np.array(y) - 206) / 454 * 720)
            raw_file = gt['raw_file']    

            
            gt_lanes = []
            for i in range(2):
                x = np.array(_gt_lanes[i])
                idx = x > -2
                x = (x[idx] - 106) / 952 * 1280
                x = InterpolatedUnivariateSpline(y[idx], x, k=1, ext='zeros')(y_samples).astype(int)
                x = np.where(x == 0, -2, x)
                gt_lanes.append(x.tolist())
        
            #frame = int(path.split('/')[-1][:-4])

            pred_path = BASE_DIR + f'logs_more_{model_type}/scb{sc_n}/replay/model_lane_pred.pkl'
            lanes = np.load(pred_path, allow_pickle=True)

            left_lane, right_lane = lanes[frame][0], lanes[frame][1]        

            left_lane[:, 0] = (left_lane[:, 0] - 106) / 952 * 1280
            left_lane[:, 1] = (left_lane[:, 1] - 206) / 454 * 720
            right_lane[:, 0] = (right_lane[:, 0] - 106) / 952 * 1280
            right_lane[:, 1] = (right_lane[:, 1] - 206) / 454 * 720
         
            idx = np.argsort(left_lane[:, 1])
            left_lane = left_lane[idx]
            idx = np.argsort(right_lane[:, 1])
            right_lane = right_lane[idx]

            if left_lane.shape[0] > 1:
                left_lane_x = InterpolatedUnivariateSpline(left_lane[:, 1], left_lane[:, 0], k=min(1, len(left_lane) - 1))(y_samples)
            elif  left_lane.shape[0] == 0:
                left_lane_x = np.ones_like(y_samples) * -2
            else:
                left_lane_x = np.ones_like(y_samples) * left_lane[0, 0]
            if right_lane.shape[0] > 1:
                right_lane_x = InterpolatedUnivariateSpline(right_lane[:, 1], right_lane[:, 0], k=min(1, len(right_lane) - 1))(y_samples)
            elif  right_lane.shape[0] == 0:
                right_lane = np.ones_like(y_samples) * -2
            else:
                right_lane_x = np.ones_like(y_samples) * right_lane[0, 0]
            
            
            left_lane_x[np.array(gt_lanes[0]) == -2] = -2
            right_lane_x[np.array(gt_lanes[1]) == -2] = -2

            pred_lanes = [left_lane_x, right_lane_x]

            #np.random.shuffle(pred_lanes)
            # Overall Accuracy, False Positive Rate, False Negative Rate
            acc, _fp, _fn = LaneEval.bench(pred_lanes, gt_lanes, y_samples, run_time)

            fp = _fp * 2
            tp = 2 - fp 
            fn = _fn * 2
            f1 = tp / (tp + (fn + (2 - tp)) * 0.5)
            res.append([sc_n, frame, acc, f1])
            '''
            gt_lanes_vis = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in gt_lanes]
            pred_lanes_vis = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in pred_lanes]
            
            img_vis = camera2model(all_data[frame].astype(np.uint8))

            for lane in gt_lanes_vis:
                cv2.polylines(img_vis, np.int32([lane]), isClosed=False, color=(0,255,0), thickness=2)
            for lane in pred_lanes_vis:
                cv2.polylines(img_vis, np.int32([lane]), isClosed=False, color=(255,0,0), thickness=2)

            imgs.append(img_vis)
            plt.imshow(img_vis)
            '''
            

    print(len(res))
    #plt.figure(figsize=(18, 12))
    #t = cv2.vconcat([cv2.hconcat(imgs[i:i+5]) for i in range(0, len(imgs), 5)])
    #plt.title(model_type)
    #plt.imshow(t)
    pd.DataFrame(res, columns=['scn', 'frame', 'ACC', 'F1']).to_csv(f'df_acc_attack_{model_type}.csv', index=False)
    
    tmp = pd.DataFrame(res, columns=['scn', 'frame', 'ACC', 'F1']).drop(['scn', 'frame'], axis=1).mean(axis=0)
    tmp['model_type'] = model_type
    df_res.append(tmp)
pd.concat(df_res, axis=1).T.set_index('model_type')

laneatt
2000
scnn
2000
ultrafast
2000
polylanenet
2000


,ACC,F1
model_type,,
laneatt,0.51346,0.1405
scnn,0.678183,0.312
ultrafast,0.601054,0.2055
polylanenet,0.587272,0.13425
